In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from qkeras import QActivation,QConv2D,QDense,quantized_bits
import qkeras
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import *
from telescope import *
from utils import *


In [18]:
encoder([tf.random.normal([2000, 9, 9, 1]),tf.random.normal([2000,1]),tf.random.normal([2000,1])])

2023-08-10 04:20:04.355011: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


<tf.Tensor: shape=(2000, 18), dtype=float32, numpy=
array([[-0.6484375 , -0.8828125 ,  1.484375  , ...,  1.203125  ,
        -0.22449946,  0.72620314],
       [-0.0859375 , -1.5       ,  1.9921875 , ..., -0.25      ,
         0.41269925,  1.3830566 ],
       [-0.890625  ,  0.328125  ,  1.890625  , ...,  0.1171875 ,
         1.1381445 , -1.1917657 ],
       ...,
       [ 0.34375   , -1.8046875 ,  1.890625  , ...,  0.9453125 ,
        -0.2736338 ,  0.5584123 ],
       [ 0.765625  , -0.5546875 , -0.0625    , ...,  0.640625  ,
        -0.8595276 , -0.51018775],
       [-0.5       , -0.875     , -0.046875  , ...,  1.8046875 ,
        -1.0860186 ,  1.8798432 ]], dtype=float32)>

In [19]:
encoder([tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype),
                                              tf.TensorSpec(model.inputs[1].shape, model.inputs[1].dtype),
                                              tf.TensorSpec(model.inputs[2].shape, model.inputs[2].dtype)])

ValueError: Exception encountered when calling layer 'conv2d' (type QConv2D).

Attempt to convert a value (TensorSpec(shape=(2000, 9, 9, 1), dtype=tf.float32, name=None)) with an unsupported type (<class 'tensorflow.python.framework.tensor.TensorSpec'>) to a Tensor.

Call arguments received by layer 'conv2d' (type QConv2D):
  • inputs=TensorSpec(shape=(2000, 9, 9, 1), dtype=tf.float32, name=None)

In [23]:
model = encoder
@tf.function
def full_model(x,y,z):
    return model([x,y,z])

full_model = full_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype),
                                              tf.TensorSpec(model.inputs[1].shape, model.inputs[1].dtype),
                                              tf.TensorSpec(model.inputs[2].shape, model.inputs[2].dtype)
                                              )

In [1]:
import tensorflow as tf

# Define a function with multiple input arguments
@tf.function
def add_and_multiply(a, b, c):
    return (a + b) * c

# Create representative input tensors
a_rep = tf.constant(2.0)
b_rep = tf.constant(3.0)
c_rep = tf.constant(4.0)

# Get the concrete function
concrete_func = add_and_multiply.get_concrete_function(a_rep, b_rep, c_rep)

# Test the concrete function
result = concrete_func(a_rep, b_rep, c_rep)
print("Result:", result.numpy())


2023-08-10 04:16:50.976736: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 04:16:53.114599: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Result: 20.0


2023-08-10 04:16:57.924537: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10823 MB memory:  -> device: 0, name: Tesla P100-PCIE-12GB, pci bus id: 0000:65:00.0, compute capability: 6.0


In [4]:
numbers_list = [
    0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 
    25, 28, 29, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 44, 45, 48, 49, 50, 51, 
    52, 53, 54, 55, 56, 57, 60, 61, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 76, 
    77, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 92, 93, 96, 97, 98, 99, 100, 101, 
    102, 103, 104, 105, 108, 109, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 124, 125
]



In [5]:
#https://github.com/rohanshenoy/Ecoder/blob/628fef43f1d5803999b4f30f7157f284fdb7c816/qDenseCNN.py#L136
def GetQbits(self, inp, keep_negative=1):
    print("Setting bits {} {} with keep negative = {}".format(inp['total'], inp['integer'], keep_negative))
    b =  qkeras.quantized_bits(bits=inp['total'], integer=inp['integer'], keep_negative=keep_negative, alpha=1)
    print('max = %s, min = %s'%(b.max(),b.min()))
    print('str representation:%s'%(str(b)))
    print('config = ',b.get_config())
    return b

In [6]:
def mean_mse_loss(y_true, y_pred):
    y_true = tf.matmul(K.reshape(y_true,(-1,81)),remap_9x9_matrix)
    y_pred = tf.matmul(K.reshape(y_pred,(-1,81)),remap_9x9_matrix)
    # Calculate the squared difference between predicted and target values
    squared_diff = tf.square(y_pred - y_true)

    # Calculate the MSE per row (reduce_mean along axis=1)
    mse_per_row = tf.reduce_mean(squared_diff, axis=1)

    # Take the mean of the MSE values to get the overall MSE loss
    mean_mse_loss = tf.reduce_mean(mse_per_row)

    return mean_mse_loss

In [7]:
batch = 2000

#Specs
n_kernels = 8
n_encoded=16
conv_weightBits  = 6 
conv_biasBits  = 6 
dense_weightBits  = 6 
dense_biasBits  = 6 
encodedBits = 9
CNN_kernel_size = 3





input_enc = Input(batch_shape=(batch,9,9, 1))
sum_input = Input(batch_shape=(batch,1))
eta = Input(batch_shape =(batch,1))
x = QActivation(qkeras.quantized_bits(bits = 8, integer = 1),name = 'input_quantization')(input_enc)

x = QConv2D(n_kernels,
            CNN_kernel_size, 
#             padding='same',
            strides=2,
            kernel_quantizer=quantized_bits(bits=conv_weightBits,integer=0,keep_negative=1,alpha=1),
            bias_quantizer=quantized_bits(bits=conv_biasBits,integer=0,keep_negative=1,alpha=1),
            name="conv2d")(input_enc)
x = QActivation("quantized_relu(bits=8,integer=1)", name="act")(x)

x = Flatten()(x)

x = QDense(n_encoded, 
           kernel_quantizer=quantized_bits(bits=dense_weightBits,integer=0,keep_negative=1,alpha=1),
           bias_quantizer=quantized_bits(bits=dense_biasBits,integer=0,keep_negative=1,alpha=1),
           name="dense")(x)
x = QActivation(qkeras.quantized_bits(bits = 9, integer = 1),name = 'latent_quantization')(x)

x = concatenate([x,sum_input,eta],axis=1)

latent = x

input_dec = Input(batch_shape=(batch,18))
y = Dense(24)(input_dec)
y = ReLU()(y)
y = Dense(64)(y)
y = ReLU()(y)
y = Dense(128)(y)
y = ReLU()(y)
y = Reshape((4, 4, 8))(y)
y = Conv2DTranspose(1, (3, 3), strides=(2, 2))(y)
y = ReLU()(y)

# y = tf.keras.activations.sigmoid(y)
recon = y




encoder = keras.Model([input_enc,sum_input,eta], latent, name="encoder")
decoder = keras.Model([input_dec], recon, name="decoder")

cae = Model(
    inputs=[input_enc,sum_input,eta],
    outputs=decoder([encoder([input_enc,sum_input,eta])]),
    name="cae"
)
#telescopeMSE9x9
#mean_mse_loss
cae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-5,weight_decay = 0.00025), loss=mean_mse_loss)
cae.summary()


Model: "cae"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(2000, 9, 9, 1)]            0         []                            
                                                                                                  
 input_2 (InputLayer)        [(2000, 1)]                  0         []                            
                                                                                                  
 input_3 (InputLayer)        [(2000, 1)]                  0         []                            
                                                                                                  
 encoder (Functional)        (2000, 18)                   2144      ['input_1[0][0]',             
                                                                     'input_2[0][0]',           

In [10]:
dt = pd.read_csv('../ECON_AE_Development/AE_Data/1.csv').values

In [ ]:
data_tensor = tf.convert_to_tensor(np.concatenate(data_list), dtype=tf.float32)
data_tensor = data_tensor[0:500000]
train_size = int(0.8 * len(data_tensor))
test_size = len(data_tensor) - train_size

# Split the data into training and test sets
train_data, test_data = tf.split(data_tensor, [train_size, test_size], axis=0)

# Extract specific tensors
if normalize:
    train_sum_calcq = tf.expand_dims(tf.reduce_sum(train_data[:, 0:48], axis=1), axis=1)
    train_data = tf.boolean_mask(train_data,tf.squeeze(train_sum_calcq,axis=-1) != 0)
    train_sum_calcq = tf.boolean_mask(train_sum_calcq,tf.squeeze(train_sum_calcq,axis=-1) != 0)
    train_wafers = expand_tensor(train_data[:, 0:48]/train_sum_calcq)
else:
    train_sum_calcq = tf.expand_dims(tf.reduce_sum(train_data[:, 0:48], axis=1), axis=1)
    train_wafers = expand_tensor(train_data[:, 0:48])

In [13]:
if tf.test.is_gpu_available():
    print("GPU is available.")
    print("GPU device name:", tf.test.gpu_device_name())
else:
    print("GPU is not available.")



GPU is available.
GPU device name: /device:GPU:0


2023-08-01 16:26:33.862632: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /device:GPU:0 with 10823 MB memory:  -> device: 0, name: Tesla P100-PCIE-12GB, pci bus id: 0000:65:00.0, compute capability: 6.0
2023-08-01 16:26:33.866730: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /device:GPU:0 with 10823 MB memory:  -> device: 0, name: Tesla P100-PCIE-12GB, pci bus id: 0000:65:00.0, compute capability: 6.0


In [4]:
def load_data(nfiles,batchsize, normalize = False):
    ecr = np.vectorize(encode)
    data_list = []

    for i in range(nfiles):
        if i == 0:
            dt = pd.read_csv('../ECON_AE_Development/AE_Data/1.csv').values
        else:
            dt_i = pd.read_csv(f'../ECON_AE_Development/AE_Data/{i+1}.csv').values
            dt = np.vstack([dt, dt_i])

        data_list.append(dt)

    data_tensor = tf.convert_to_tensor(np.concatenate(data_list), dtype=tf.float32)
    data_tensor = data_tensor[0:500000]
    train_size = int(0.8 * len(data_tensor))
    test_size = len(data_tensor) - train_size

    # Split the data into training and test sets
    train_data, test_data = tf.split(data_tensor, [train_size, test_size], axis=0)

    # Extract specific tensors
    if normalize:
        train_sum_calcq = tf.expand_dims(tf.reduce_sum(train_data[:, 0:48], axis=1), axis=1)
        train_data = tf.boolean_mask(train_data,tf.squeeze(train_sum_calcq,axis=-1) != 0)
        train_sum_calcq = tf.boolean_mask(train_sum_calcq,tf.squeeze(train_sum_calcq,axis=-1) != 0)
        train_wafers = expand_tensor(train_data[:, 0:48]/train_sum_calcq)
    else:
        train_sum_calcq = tf.expand_dims(tf.reduce_sum(train_data[:, 0:48], axis=1), axis=1)
        train_wafers = expand_tensor(train_data[:, 0:48])
    
    

    if normalize:
        test_sum_calcq = tf.expand_dims(tf.reduce_sum(test_data[:, 0:48], axis=1), axis=1)
        test_data = tf.boolean_mask(test_data,tf.squeeze(test_sum_calcq,axis=-1) != 0)
        test_sum_calcq = tf.boolean_mask(test_sum_calcq,tf.squeeze(test_sum_calcq,axis=-1) != 0)
        test_wafers = expand_tensor(test_data[:, 0:48]/test_sum_calcq)
    else:
        test_sum_calcq = tf.expand_dims(tf.reduce_sum(test_data[:, 0:48], axis=1), axis=1)
        test_wafers = expand_tensor(test_data[:, 0:48])
    
    ''' 
    
    Quantize Sum CalcQ with 5 exp bits and 4 mant bits
    
    Taking the log here is effectivel taking the log on readout. It makes no change to the quantization,
    just a convenient setup for keras.
    
    '''
    train_sum_calcq = train_sum_calcq.numpy().astype(int)
    train_sum_calcq = ecr(train_sum_calcq, dropBits=0, expBits=5, mantBits=4, roundBits=False, asInt=True)
    train_sum_calcq = tf.math.log(tf.constant(train_sum_calcq,dtype = tf.float64))
    train_eta = tf.expand_dims(train_data[:, -2], axis=1)
    
    test_sum_calcq = test_sum_calcq.numpy().astype(int)
    test_sum_calcq = ecr(test_sum_calcq, dropBits=0, expBits=5, mantBits=4, roundBits=False, asInt=True)
    test_sum_calcq = tf.math.log(tf.constant(test_sum_calcq,dtype = tf.float64))
    test_eta = tf.expand_dims(test_data[:, -2], axis=1)

    # Create data loaders for training and test data
    train_dataset = tf.data.Dataset.from_tensor_slices((train_wafers, train_sum_calcq, train_eta))
    train_loader = train_dataset.batch(batchsize).shuffle(buffer_size=train_size).prefetch(buffer_size=tf.data.AUTOTUNE)

    test_dataset = tf.data.Dataset.from_tensor_slices((test_wafers, test_sum_calcq, test_eta))
    test_loader = test_dataset.batch(batchsize).shuffle(buffer_size=test_size).prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_loader, test_loader


In [5]:
def expand_tensor(input_tensor):
    arrange = np.array([28,29,30,31,0,4,8,12,
                         24,25,26,27,1,5,9,13,
                         20,21,22,23,2,6,10,14,
                         16,17,18,19,3,7,11,15,
                         47,43,39,35,35,34,33,32,
                         46,42,38,34,39,38,37,36,
                         45,41,37,33,43,42,41,40,
                             44,40,36,32,47,46,45,44])
    arrMask = np.array([1,1,1,1,1,1,1,1,
                        1,1,1,1,1,1,1,1,
                        1,1,1,1,1,1,1,1,
                        1,1,1,1,1,1,1,1,
                        1,1,1,1,0,0,0,0,
                        1,1,1,1,0,0,0,0,
                        1,1,1,1,0,0,0,0,
                        1,1,1,1,0,0,0,0,])
   
    inputdata = tf.reshape(tf.gather(input_tensor, arrange,axis =1), (input_tensor.shape[0],8, 8, 1))
#     inputdata *= tf.cast(tf.reshape(arrMask, (1, 8, 8)), dtype=inputdata.dtype)

    paddings = [(0, 0), (0, 1), (0, 1), (0, 0)]
    padded_tensor = tf.pad(inputdata, paddings, mode='CONSTANT', constant_values=0)

    return padded_tensor


In [6]:
train_loader, test_loader = load_data(1,batch,normalize = True)

In [9]:
remap_9x9 = [
    4, 13, 22, 31, 5, 14, 23, 32, 6, 15, 24, 33, 7, 16, 25, 34,
    27, 28, 29, 30, 18, 19, 20, 21, 9, 10, 11, 12, 0, 1, 2, 3,
    66, 57, 48, 40, 65, 56, 50, 49, 64, 60, 59, 58, 70, 69, 68, 67
]

remap_9x9_matrix = np.zeros(48*81,dtype=np.float32).reshape((81,48))

for i in range(48): 
    remap_9x9_matrix[remap_9x9[i],i] = 1
# def telescopeMSE9x9(y_true,y_pred):
#     return telescopeMSE2(tf.matmul(K.reshape(y_true,(-1,81)),remap_9x9_matrix),
#                          tf.matmul(K.reshape(y_pred,(-1,81)),remap_9x9_matrix))

In [83]:
dif = tf.matmul(K.reshape(cae([wafers, sum_calcq, eta]),(-1,81)),remap_9x9_matrix)-tf.matmul(K.reshape(wafers,(-1,81)),remap_9x9_matrix)


In [72]:
tf.reduce_sum(dif,axis = [1])/sum_calcq

<tf.Tensor: shape=(2000, 2000), dtype=float32, numpy=
array([[0.88447297, 0.22068278, 0.7167299 , ..., 1.0648726 , 0.8572591 ,
        0.60360783],
       [0.51027286, 0.12731698, 0.41349798, ..., 0.6143496 , 0.49457258,
        0.34823528],
       [6.6335473 , 1.6551208 , 5.375474  , ..., 7.986545  , 6.4294434 ,
        4.5270586 ],
       ...,
       [1.8952992 , 0.47289166, 1.5358497 , ..., 2.2818701 , 1.8369838 ,
        1.2934453 ],
       [1.6583868 , 0.4137802 , 1.3438685 , ..., 1.9966363 , 1.6073608 ,
        1.1317647 ],
       [0.60304976, 0.15046553, 0.48867944, ..., 0.72604954, 0.5844948 ,
        0.4115508 ]], dtype=float32)>

In [123]:
tf.matmul(K.reshape(cae([wafers, sum_calcq, eta])[i],(-1,81)),remap_9x9_matrix)


<tf.Tensor: shape=(1, 48), dtype=float32, numpy=
array([[0.        , 0.        , 0.1350457 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.0309369 ,
        0.        , 0.        , 0.00258605, 0.        , 0.04333076,
        0.25556087, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.0372609 ,
        0.        , 0.01337619, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.18429805,
        0.        , 0.        , 0.3622913 , 0.        , 0.        ,
        0.03496099, 0.        , 0.        ]], dtype=float32)>

In [124]:
tf.matmul(K.reshape(wafers[i],(-1,81)),remap_9x9_matrix)

<tf.Tensor: shape=(1, 48), dtype=float32, numpy=
array([[0.        , 0.        , 0.08648649, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.16216215, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.2       , 0.16216215, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.38918918, 0.        , 0.        ,
        0.        , 0.        , 0.        ]], dtype=float32)>

In [9]:
# cae.compile(optimizer=tf.keras.optimizers.Lion(learning_rate = 5e-3,weight_decay = 0.00025), loss=telescopeMSE9x9)

nepochs = 50
for epoch in range(nepochs):
    total_loss = 0
    for wafers, sum_calcq, eta in train_loader:
        # Train the CAE model on each batch

        if wafers.shape[0] != batch:
            break
        loss = cae.train_on_batch([wafers, sum_calcq, eta], wafers)
        total_loss =total_loss + loss
        
    print(f"Epoch {epoch+1}/{nepochs}, Loss: {total_loss/(len(train_loader)*batch)}")


    

2023-08-01 15:09:40.825128: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape ingradient_tape/cae/encoder/act/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Epoch 1/50, Loss: 6.043301221216097e-06
Epoch 2/50, Loss: 1.492909830994904e-07
Epoch 3/50, Loss: 1.7512552556581796e-06
Epoch 4/50, Loss: 9.572969051077962e-08
Epoch 5/50, Loss: 4.921620690729469e-06
Epoch 6/50, Loss: 4.025467851897702e-06
Epoch 7/50, Loss: 2.377213683212176e-06


KeyboardInterrupt: 

In [ ]:
def load_data(args):
    data_list = []

    for i in range(args.nfiles):
        if i == 0:
            dt = pd.read_csv('../ECON_AE_Development/AE_Data/1.csv').values
        else:
            dt_i = pd.read_csv(f'../ECON_AE_Development/AE_Data/{i+1}.csv').values
            dt = np.vstack([dt, dt_i])

        data_list.append(dt)

    data_tensor = tf.convert_to_tensor(np.concatenate(data_list), dtype=tf.float32)

    train_size = int(0.8 * len(data_tensor))
    test_size = len(data_tensor) - train_size

    # Split the data into training and test sets
    train_data, test_data = tf.split(data_tensor, [train_size, test_size], axis=0)

    # Extract specific tensors
    train_wafers = train_data[:, 0:48]
    train_sum_calcq = tf.reduce_sum(train_data[:, 0:48], axis=1)
    train_eta = train_data[:, -2]

    test_wafers = test_data[:, 0:48]
    test_sum_calcq = tf.reduce_sum(test_data[:, 0:48], axis=1)
    test_eta = test_data[:, -2]

    # Create data loaders for training and test data
    train_dataset = tf.data.Dataset.from_tensor_slices((train_wafers, train_sum_calcq, train_eta))
    train_loader = train_dataset.batch(args.batchsize).shuffle(buffer_size=train_size).prefetch(buffer_size=tf.data.AUTOTUNE)

    test_dataset = tf.data.Dataset.from_tensor_slices((test_wafers, test_sum_calcq, test_eta))
    test_loader = test_dataset.batch(args.batchsize).shuffle(buffer_size=test_size).prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_loader, test_loader
